# Regressão 01 - tarefa 03 - transformações em X e Y

Carregue os pacotes necessários e a base de gorjetas.

### I. Modelo no valor da gorjeta

1. Crie a matriz de design (e a matriz y) utilizando o Patsy, para um modelo em ```tip```, explicada por ```sex, smoker, diner e net_bill```.  
2. Remova as variáveis não significantes.  
3. observe o gráfico de resíduos em função de ```net_bill```  
4. teste transformar ```net_bill``` no log e um polinômio. Escolha o melhor modelo.

In [197]:
import pandas as pd
import seaborn as sns
import patsy
import statsmodels.api as sm
import numpy as np
import itertools

In [198]:
# carrega a base de dados

tips = sns.load_dataset('tips')
tips['tip_pct'] = tips['tip']/(tips['total_bill'] - tips['tip'])
tips['net_bill'] = tips['total_bill'] - tips['tip']

tips.head()

In [199]:
# ajustando o modelo

def modelo(formula: str, df):
    
    
    y_m, X_m = patsy.dmatrices(formula, data=df)
    modelo_m = sm.OLS(y, X).fit()
    #salva os coeficientes e os valores de p associados e o valores de R²
    p_values = modelo_m.pvalues
    coef = modelo_m.params
    r_2_values = modelo_m.rsquared
    
    # cria um df com as informações acima
    df = pd.DataFrame({'coef': coef, 'p_values': p_values, 'R² values': r_2_values})
    print(df)

In [200]:
formulas = (['tip ~ sex + smoker + time + net_bill', 
             'tip ~ smoker + time + net_bill',
             'tip ~ sex + smoker + time',
             'tip ~ sex + smoker + net_bill',
             'tip ~ time + net_bill',
             'tip ~ smoker + net_bill',
             'tip ~ smoker + time',
             'tip ~ sex + smoker',
             'tip ~ sex + net_bill',
             'tip ~ smoker',
             'tip ~ time',
             'tip ~ net_bill',
             'tip ~ sex'])

for formula in formulas:
    print(formula)
    modelo(formula, tips)

A varíavel net_bill é que possui melhor descrição, e pode ser usada sozinha.

In [201]:
y, X = patsy.dmatrices('tip ~ net_bill', data=tips)
modelo = sm.OLS(y, X).fit()

# usando residplot
sns.residplot(data=tips, x='net_bill', y=modelo.resid)

In [202]:
formulas = ['tip ~ np.log(net_bill)', 'tip ~ net_bill + np.power(net_bill,2)']

for formula in formulas:
    print(formula)
    modelo(formula, tips)

O ajuste de net_bill por polinômios fornece um R² melhor que o do log. Porém, a variável sem ajustes já é suficiente para explicar.

### II. Modelo no valor do percentual da gorjeta

1. Crie a matriz de design (e a matriz y) utilizando o Patsy, para um modelo no log de ```tip_pct```, explicado por ```sex, smoker, diner e net_bill```.
2. Remova as variáveis não significantes.
3. Observe o gráfico de resíduos em função de ```net_bill```
4. Teste transformar ```net_bill``` no log e um polinômio. Escolha o melhor modelo.
5. Do modelo final deste item, calcule o $R^2$ na escala de ```tip``` (sem o log). Compare com o modelo do item 1. Qual tem melhor coeficiente de determinação?

In [203]:
# restringindo os valores de até 70% para a porcentagem

tips_1 = tips[tips['tip_pct'] <= 0.7]

In [204]:
formulas = (['tip_pct ~ sex + smoker + time + net_bill', 
             'tip_pct ~ smoker + time + net_bill',
             'tip_pct ~ sex + smoker + time',
             'tip_pct ~ sex + smoker + net_bill',
             'tip_pct ~ time + net_bill',
             'tip_pct ~ smoker + net_bill',
             'tip_pct ~ smoker + time',
             'tip_pct ~ sex + smoker',
             'tip_pct ~ sex + net_bill',
             'tip_pct ~ smoker',
             'tip_pct ~ time',
             'tip_pct ~ net_bill',
             'tip_pct ~ sex'])

for formula in formulas:
    print(formula)
    modelo(formula, tips_1)

In [205]:
y, X = patsy.dmatrices('tip_pct ~ smoker + net_bill', data=tips_1)
modelo = sm.OLS(y, X).fit()

# usando residplot
sns.residplot(data=tips_1 , x='net_bill', y=modelo.resid)

In [206]:
formulas = ['tip_pct ~ np.log(net_bill)', 'tip_pct ~ net_bill + np.power(net_bill,2)']

for formula in formulas:
    print(formula)
    modelo(formula, tips_1)

O modelo log se saiu melhor.

In [207]:
y, X = patsy.dmatrices('tip_pct ~ np.log(net_bill)', data=tips_1)
modelo = sm.OLS(y, X).fit()

# usando residplot
sns.residplot(data=tips_1, x='net_bill', y=modelo.resid)

In [208]:
print(f'R² do modelo com log: {modelo.rsquared: .2f}')

O modelo que usa a tip como variável resposta possui um R² maior.

### III. Previsão de renda

Vamos trabalhar a base que você vai usar no projeto do final deste ciclo.

Carregue a base ```previsao_de_renda.csv```.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Ajuste um modelo de regressão linear simples para explicar ```renda``` como variável resposta, por ```tempo_emprego``` como variável explicativa. Observe que há muitas observações nessa tabela. Utilize os recursos que achar necessário.
2. Faça uma análise de resíduos. Com os recursos vistos neste módulo, como você melhoraria esta regressão?
3. Ajuste um modelo de regressão linear múltipla para explicar ```renda``` (ou uma transformação de ```renda```) de acordo com as demais variáveis.
4. Remova as variáveis não significantes e ajuste novamente o modelo. Interprete os parâmetros
5. Faça uma análise de resíduos. Avalie a qualidade do ajuste.

In [ ]:
previsao_renda_df = pd.read_csv('previsao_de_renda.csv')

In [ ]:
previsao_renda_df.shape

In [ ]:
previsao_renda_df.isna().sum()

In [ ]:
# dropando os valores vazios:
previsao_renda_df.dropna(inplace=True)

In [ ]:
previsao_renda_df.dtypes

In [ ]:
prev_renda_dummies = pd.get_dummies(previsao_de_renda.csv, 
                            columns=['posse_de_veiculo', 'posse_de_imovel', 'mau'], 
                            drop_first=True).drop(columns=['ano', 'mes'])

In [ ]:
# ajusta o modelo

modelo('renda ~ tempo_emprego', df=previsao_renda_df)